# Прогнозирование температуры звезды

## Описание проекта
**Задача от обсерватории «Небо на ладони»**: разработать нейросеть для определения температуры на поверхности обнаруженных звёзд.

### Цель проекта
Создать модель машинного обучения, которая будет предсказывать абсолютную температуру звезды на основе её характеристик.

---

## Традиционные методы расчёта температуры
Обычно учёные используют:
1. **Закон смещения Вина**
2. **Закон Стефана-Больцмана** 
3. **Спектральный анализ**

Каждый метод имеет преимущества и недостатки. ML-подход должен стать более точной и удобной альтернативой.

---

## Данные для обучения
В базе обсерватории есть характеристики **240 изученных звёзд**:

### Признаки звезды:
| Признак | Описание |
|---------|-----------|
| **Относительная светимость** | L/Lo - светимость звезды относительно Солнца |
| **Относительный радиус** | R/Ro - радиус звезды относительно радиуса Солнца |
| **Абсолютная звёздная величина** | Mv - физическая величина, характеризующая блеск звезды |
| **Звёздный цвет** | white, red, blue, yellow, yellow-orange и др. |
| **Тип звезды** | Категориальный признак |

### Типы звезд:
| Тип звезды | Числовой код |
|------------|--------------|
| Коричневый карлик | 0 |
| Красный карлик | 1 |
| Белый карлик | 2 |
| Звёзды главной последовательности | 3 |
| Сверхгигант | 4 |
| Гипергигант | 5 |

### Целевая переменная:
- **Абсолютная температура T(K)** - температура на поверхности звезды в Кельвинах

# Импортируем библиотеки

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
from IPython.display import HTML, display
import seaborn as sns
import re
import warnings
import logging
import optuna

# === Функции проекта ===
from func import (
    EarlyStoppingCallback,
    check_data,
    drop_duplicated,
    parse_category_ids,
    plot_combined,
    calc_target_correlations,
    suggest_param,
    agg_period
)

# === Настройки ===
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings(
    "ignore",
    category=RuntimeWarning,
    module="sklearn.feature_selection._univariate_selection",
)
logging.getLogger("sklearn").setLevel(logging.ERROR)
optuna.logging.set_verbosity(optuna.logging.WARNING)
pd.set_option("display.max_columns", None)

xgb_params = {"verbosity": 0}

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

warnings.filterwarnings("ignore", category=UserWarning, module="shap")

# Константы

In [9]:
# зафиксируем константы
RANDOM_STATE = 20
TEST_SIZE = 0.25
N_JOBS = -1
N_ITER = 10 # число итераций для перебора и поиска лучших параметров
N_CROSS_VALL = 3

# Загрузка данных

In [10]:
df = pd.read_csv("../data/6_class.csv")

# Анализ и предобработка данных

In [11]:
df = check_data(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   unnamed:_0               240 non-null    int64  
 1   temperature_(_k)         240 non-null    int64  
 2   luminosity(_l/_lo)       240 non-null    float64
 3   radius(_r/_ro)           240 non-null    float64
 4   absolute_magnitude(_mv)  240 non-null    float64
 5   star_type                240 non-null    int64  
 6   star_color               240 non-null    object 
dtypes: float64(3), int64(3), object(1)
memory usage: 13.3+ KB


None

,unnamed:_0,temperature_(_k),luminosity(_l/_lo),radius(_r/_ro),absolute_magnitude(_mv),star_type,star_color
139,139,3541,0.001300,0.2560,14.33,1,red
224,224,16787,246730.000000,62.0000,-6.35,4,blue
119,119,3780,200000.000000,1324.0000,-10.70,5,red
180,180,2831,0.000231,0.0915,16.21,0,red
182,182,3419,0.000245,0.1260,17.56,0,red


Series([], dtype: int64)

None

,count,mean,std,min,25%,50%,75%,max
unnamed:_0,240.0,119.500000,69.426220,0.00000,59.750000,119.5000,179.2500,239.00
temperature_(_k),240.0,10497.462500,9552.425037,1939.00000,3344.250000,5776.0000,15055.5000,40000.00
luminosity(_l/_lo),240.0,107188.361635,179432.244940,0.00008,0.000865,0.0705,198050.0000,849420.00
radius(_r/_ro),240.0,237.157781,517.155763,0.00840,0.102750,0.7625,42.7500,1948.50
absolute_magnitude(_mv),240.0,4.382396,10.532512,-11.92000,-6.232500,8.3130,13.6975,20.06
star_type,240.0,2.500000,1.711394,0.00000,1.000000,2.5000,4.0000,5.00


Пропусков нет, но есть цвет звезды - проверим

In [12]:
df['star_color'].unique()

array(['red', 'blue white', 'white', 'yellowish white',
       'pale yellow orange', 'blue', 'blue-white', 'whitish',
       'yellow-white', 'orange', 'white-yellow', 'yellowish',
       'orange-red'], dtype=object)

Так и оказалось, заполняли кто как хочет... приведем неявные дубликаты к общему виду и немного наведем красоты

In [13]:
color_mapping = {
    'blue': 'blue',
    'blue white': 'blue', 
    'blue-white': 'blue',
    'whitish': 'blue',
    
    'white': 'white',
    'yellowish white': 'white',
    
    'yellow-white': 'yellow',
    'white-yellow': 'yellow',
    'yellowish': 'yellow',
    'pale yellow orange': 'yellow',
    
    'orange': 'orange',
    
    'red': 'red',
    'orange-red': 'red'
}

df['star_color'] = df['star_color'].str.strip().str.lower().replace(color_mapping)

In [ ]:
# и теперь проверим явные дубликаты после корректировок
df = drop_duplicated(df)

np.int64(0)

'Дубликаты отсутствуют'

# Построение базовой нейронной сети

# Улучшение нейронной сети

# Выводы